In [7]:
import numpy as np

In [6]:
def normalFluxJacobian2D(H,Hux,Huy,g,nx,ny):

    ## return the normal flux jacobian (2-D)

    a = np.sqrt(g*H)
    ux = Hux/H
    uy = Huy/H

    Fn = np.zeros( (3,3) )

    Fn[0,:] = ( 0., nx, ny )
    Fn[1,:] = ( (a**2 - ux**2)*nx - ux*uy*ny , 2.*ux*nx + uy*ny, ux*ny )
    Fn[2,:] = ( -ux*uy*nx + (a**2-uy**2)*ny, uy*nx, ux*nx + 2.*uy*ny )

    return Fn

In [33]:
def eigenDecomp1D(H,Hux,g):
    
    ## return the eigendecomposition of the flux jacobian (1-D)
    
    ux = Hux/H

    a = np.sqrt(H*g)
    
    lam = np.zeros(2)
    lam[0] = ux - a
    lam[1] = ux + a
    
    rightEV = np.zeros( (2,2) )
    leftEV  = np.zeros( (2,2) )
    
    rightEV[:,0] = ( 1., ux-a )
    rightEV[:,1] = ( 1., ux+a )
    
    leftEV[0,:] = ( ux+a, -1. )
    leftEV[1,:] = ( a-ux,  1. )
    leftEV /= 2.*a
    
    return lam, leftEV, rightEV

In [34]:
def eigenDecomp2D(H,Hux,Huy,g,nx,ny):
    
    ## return the eigendecomposition of the flux jacobian (2-D)
    
    ux = Hux/H
    uy = Huy/H
    vn = ux*nx + uy*ny

    a = np.sqrt(g*H)
    
    lam = np.zeros(3)
    
    lam[0] = vn + a
    lam[1] = vn
    lam[2] = vn - a
    
    rightEV = np.zeros( (3,3) )
    leftEV  = np.zeros( (3,3) )
    
    rightEV[:,0] = ( 1., ux + a*nx, uy + a*ny )
    rightEV[:,1] = ( 0., -a*ny, a*nx )
    rightEV[:,2] = ( 1., ux - a*nx, uy - a*ny )
    
    leftEV[0,:] = 1./(2.*a)*np.array( [ a-vn, nx, ny ] )
    leftEV[1,:] = 1./(2.*a)*np.array( [ 2*(ux*ny - uy*nx), -2.*ny, 2.*nx ] )
    leftEV[2,:] = 1./(2.*a)*np.array( [ a + vn, -nx, -ny ] )
    
    return lam, leftEV, rightEV
    

In [64]:
def generateInputOutputEigDecomp(ndim=1):
    
    rng = np.random.default_rng()
    
    H = rng.uniform(0.,20.)
    ux = rng.uniform(-10.,10.)
    uy = rng.uniform(-10.,10.)
    nx = 1.
    g = 9.81 
    
    Hux = H*ux
    Huy = H*uy

    Fn = None

    if ndim == 1:
        lam,LEV,REV = eigenDecomp1D(H,Hux,g)
    elif ndim == 2:
        nx = rng.uniform(-1.,1.)
        nyMag = np.sqrt(1. - nx**2)
        ny = nyMag * rng.choice([1.,-1.])
        lam,LEV,REV = eigenDecomp2D(H,Hux,Huy,g,nx,ny)
        Fn = normalFluxJacobian2D(H,Hux,Huy,g,nx,ny)
    
    ## Print out format so we can plop directly in code...
    
    print( f"AD Hux = {Hux}; AD H = {H}; \n" )
    
    if ndim == 2:
        print ( f"AD Huy = {Huy}; ScalarT nx = {nx}; ScalarT ny = {ny};\n" )
    
    lamstring = ""
    LEVstring = ""
    REVstring = ""
    
    for j in range(1+ndim):
        lamstring += f"lamExact({j}) = {lam[j]}; "
        for i in range(1+ndim):
            LEVstring += f"LEvExact({i},{j}) = {LEV[i,j]}; "
            REVstring += f"REvExact({i},{j}) = {REV[i,j]}; "
        LEVstring += "\n"
        REVstring += "\n"
    lamstring += "\n"
    
    print (lamstring)
    print (LEVstring)
    print (REVstring)

    ## check here that the eigendecomp works
    if Fn is not None:
        L = np.diag(lam)
        A = REV@L@LEV
        print(np.linalg.norm(A-Fn))

In [65]:
generateInputOutputEigDecomp(2)

AD Hux = -39.22846164587605; AD H = 4.039388075475017; 

AD Huy = 34.522032232062756; ScalarT nx = -0.3714635145497569; ScalarT ny = -0.9284475522927198;

lamExact(0) = 1.9675733580306787; lamExact(1) = -4.327376762535005; lamExact(2) = -10.622326883100689; 

LEvExact(0,0) = 0.8437181136985826; LEvExact(1,0) = 1.936673574640384; LEvExact(2,0) = 0.1562818863014173; 
LEvExact(0,1) = -0.029504881487160697; LEvExact(1,1) = 0.14749085131897544; LEvExact(2,1) = 0.029504881487160697; 
LEvExact(0,2) = -0.07374542565948772; LEvExact(1,2) = -0.059009762974321395; LEvExact(2,2) = 0.07374542565948772; 

REvExact(0,0) = 1.0; REvExact(1,0) = -12.049830519084452; REvExact(2,0) = 2.7018209376747606; 
REvExact(0,1) = 0.0; REvExact(1,1) = 5.844531031243971; REvExact(2,1) = -2.338344295700745; 
REvExact(0,2) = 1.0; REvExact(1,2) = -7.373141927682962; REvExact(2,2) = 14.390883000162702; 

[[ 5.40946516e-17 -3.71463515e-01 -9.28447552e-01]
 [-5.67450205e+01 -7.19913958e-01  9.01660561e+00]
 [ 1.92253595e-0

In [10]:
def stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='Roe-like'):

    ## form delta S

    S = np.array( [H,Hux,Huy] )
    Shat = np.array( [ Hhat,Huxhat,Huyhat] ) 

    deltaS = S - Shat

    ## Get eigendecomp

    Fnhat = normalFluxJacobian2D(Hhat,Huxhat,Huyhat,g,nx,ny)
    l,rv = np.linalg.eig(Fnhat)

    if method == 'max-EV':
        return np.abs(l).max()*deltaS

    ## form roe-like stabilization R | lambda | R^-1
    tau = rv@np.diag(np.abs(l))@np.linalg.inv(rv)    

    return tau@deltaS

In [19]:
def fluxes2D(H,Hux,Huy,g):

    Fx = np.array( [Hux,Hux*Hux/H + g*H*H/2.,Hux*Huy/H] )
    Fy = np.array( [Huy,Huy*Hux/H,Huy*Huy/H + g*H*H/2.] )

    return Fx,Fy

In [30]:
def generateInputOutputStabAndFluxes(ndim=2):
    
    rng = np.random.default_rng()
    
    H = rng.uniform(0.,20.)
    ux = rng.uniform(-10.,10.)
    uy = rng.uniform(-10.,10.)
    nx = 1.
    g = 9.81 
    
    Hux = H*ux
    Huy = H*uy

    Hhat = rng.uniform(0.,20.)
    uxhat = rng.uniform(-10.,10.)
    uyhat = rng.uniform(-10.,10.)

    Huxhat = Hhat*uxhat
    Huyhat = Hhat*uyhat

    nx = rng.uniform(-1.,1.)
    nyMag = np.sqrt(1. - nx**2)
    ny = nyMag * rng.choice([1.,-1.])

    stab = stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='Roe-like')
    
    ## Print out format so we can plop directly in code...
    
    print( f"vector<AD> solvals = {{{H},{Hux},{Huy}}};" )
    print( f"vector<AD> auxvals = {{{Hhat},{Huxhat},{Huyhat}}};" )
    print( f"ScalarT nxVal = {nx}; ScalarT nyVal = {ny};" )
    
    print( f"vector<AD> stabExactVals = {{{stab[0]},{stab[1]},{stab[2]}}};" )

    stab = stabTerm(H,Hux,Huy,Hhat,Huxhat,Huyhat,g,nx,ny,method='max-EV')

    print ("\n Now max EV result...\n")
    print( f"stabExactVals = {{{stab[0]},{stab[1]},{stab[2]}}};" )

    Fx,Fy = fluxes2D(H,Hux,Huy,g)

    print ("\n")
    print( f"vector<AD> FxExactVals = {{{Fx[0]}, {Fx[1]}, {Fx[2]}}};")
    print( f"vector<AD> FyExactVals = {{{Fy[0]}, {Fy[1]}, {Fy[2]}}};")

    Fx,Fy = fluxes2D(Hhat,Huxhat,Huyhat,g)

    print("")
    print( f"FxExactVals = {{{Fx[0]}, {Fx[1]}, {Fx[2]}}};")
    print( f"FyExactVals = {{{Fy[0]}, {Fy[1]}, {Fy[2]}}};")

In [29]:
generateInputOutputStabAndFluxes()

vector<AD> solvals = {11.292988634298153,-5.3214599334179,-5.340172901622864};
vector<AD> auxvals = {19.950422663930343,-24.170846656508235,119.77858155615492};
ScalarT nxVal = 0.6270388286085589; ScalarT nyVal = -0.778988002100935;
vector<AD> stabExactVals = {-145.29429977594515,968.0485432174496,-2197.4366441175353};

 Now max EV result...

stabExactVals = {-168.1823676048252,366.1748361163051,-2430.601063138548};


vector<AD> FxExactVals = {-5.3214599334179, 628.0500281064968, 2.5163857906668476};
vector<AD> FyExactVals = {-5.340172901622864, 2.5163857906668476, 628.0676948909696};

vector<AD> FxExactVals = {-24.170846656508235, 1981.5690654679377, -145.11721261736545};
vector<AD> FyExactVals = {119.77858155615492, -145.11721261736545, 2671.413035360087};
